In [1]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec, utils

In [9]:
def gen_elliptic_curve_keys(): 
    curve = ec.SECP256R1()
    private_key = ec.generate_private_key(curve)
    public_key = private_key.public_key()
    return private_key, public_key

private_key, public_key = gen_elliptic_curve_keys()



In [13]:
from cryptography.hazmat.backends import default_backend

def hash(message):
    hasher = hashes.Hash(hashes.SHA256(), backend=default_backend())
    hasher.update(message.encode())
    return hasher.finalize()

print(hash("hello").hex())


2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824


In [15]:
def sign(message: str, pk: ec.EllipticCurvePrivateKey) -> bytes:
    hash_message = hash(message)
    # telling ECDSA to not hash the message as it's already hashed
    signature = pk.sign(hash_message, ec.ECDSA(utils.Prehashed(hashes.SHA256())))
    return signature

signature = sign("hello", private_key).hex()
another_signature = sign("hello", private_key).hex()

print(signature)
print(another_signature)

3045022100f4a3f0d39b5aa0299ab641350d140a3ee61153554369912e9e465d07f6f8124302204daf85a4db9f93b971ed77b54ace8f2387bf82e791fa25ec09e1e5ef80b64fee
3046022100c9c347b9411471c7378aef2f5706cde52b567c479b3e1a54c0fe6950d393afe1022100c4fb27c2f38faf3a481dce31d85cef6e5da4fca78896891388d301a50c2f7d88


In [19]:
def verify(message: str, signature: bytes, pk: ec.EllipticCurvePublicKey) -> None:
    hash_message = hash(message)
    pk.verify(signature, hash_message, ec.ECDSA(utils.Prehashed(hashes.SHA256())))

verify("hello", bytes.fromhex(signature), public_key)


In [21]:
verify("hello, hacked", bytes.fromhex(signature), public_key)

InvalidSignature: 